In [1]:
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import ast
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
from sklearn.metrics import confusion_matrix
from mpl_toolkits.mplot3d import Axes3D
import librosa, librosa.display
import itertools

In [2]:
features = pd.read_csv('data/fma_metadata/features.csv', index_col=0, header=[0, 1, 2])
tracks = pd.read_csv('data/fma_metadata/tracks.csv', index_col=0, header=[0, 1])

In [4]:
def columns():
    feature_sizes = dict(chroma_stft=12, chroma_cqt=12, chroma_cens=12,
                         tonnetz=6, mfcc=20, rmse=1, zcr=1,
                         spectral_centroid=1, spectral_bandwidth=1,
                         spectral_contrast=7, spectral_rolloff=1)
    moments = ('mean', 'std', 'skew', 'kurtosis', 'median', 'min', 'max')

    columns = []
    for name, size in feature_sizes.items():
        for moment in moments:
            it = ((name, moment, '{:02d}'.format(i+1)) for i in range(size))
            columns.extend(it)

    names = ('feature', 'statistics', 'number')
    columns = pd.MultiIndex.from_tuples(columns, names=names)

    # More efficient to slice if indexes are sorted.
    return columns.sort_values()


def compute_features(filepath):

    features = pd.Series(index=columns(), dtype=np.float32)

    def feature_stats(name, values):
        features[name, 'mean'] = np.mean(values, axis=1)
        features[name, 'std'] = np.std(values, axis=1)
        features[name, 'skew'] = stats.skew(values, axis=1)
        features[name, 'kurtosis'] = stats.kurtosis(values, axis=1)
        features[name, 'median'] = np.median(values, axis=1)
        features[name, 'min'] = np.min(values, axis=1)
        features[name, 'max'] = np.max(values, axis=1)

    x, sr = librosa.load(filepath, sr=None, mono=True)  # kaiser_fast
    f = librosa.feature.zero_crossing_rate(x, frame_length=2048, hop_length=512)
    feature_stats('zcr', f)

    cqt = np.abs(librosa.cqt(x, sr=sr, hop_length=512, bins_per_octave=12,
                                 n_bins=7*12, tuning=None))

    f = librosa.feature.chroma_cqt(C=cqt, n_chroma=12, n_octaves=7)
    feature_stats('chroma_cqt', f)
    f = librosa.feature.chroma_cens(C=cqt, n_chroma=12, n_octaves=7)
    feature_stats('chroma_cens', f)
    f = librosa.feature.tonnetz(chroma=f)
    feature_stats('tonnetz', f)

    stft = np.abs(librosa.stft(x, n_fft=2048, hop_length=512))

    f = librosa.feature.chroma_stft(S=stft**2, n_chroma=12)
    feature_stats('chroma_stft', f)

    f = librosa.feature.rmse(S=stft)
    feature_stats('rmse', f)

    f = librosa.feature.spectral_centroid(S=stft)
    feature_stats('spectral_centroid', f)
    f = librosa.feature.spectral_bandwidth(S=stft)
    feature_stats('spectral_bandwidth', f)
    f = librosa.feature.spectral_contrast(S=stft, n_bands=6)
    feature_stats('spectral_contrast', f)
    f = librosa.feature.spectral_rolloff(S=stft)
    feature_stats('spectral_rolloff', f)

    mel = librosa.feature.melspectrogram(sr=sr, S=stft**2)

    #plt.figure(figsize=(10, 4))
    #librosa.display.specshow(librosa.power_to_db(S = mel, ref=np.max), y_axis='mel', fmax=8000, x_axis='time')
    #plt.colorbar(format='%+2.0f dB')
    #plt.title('Mel spectrogram')
    #plt.tight_layout()
    #plt.show()
    
    f = librosa.feature.mfcc(S=librosa.power_to_db(mel), n_mfcc=20)
    feature_stats('mfcc', f)
    return [features]

In [3]:
CENS = 'chroma_cens'
CQT = 'chroma_cqt'
STFT = 'chroma_stft'
MFCC = 'mfcc'
RMSE = 'rmse'
BW = 'spectral_bandwidth'
CENT = 'spectral_centroid'
CONT = 'spectral_contrast'
ROLLOFF = 'spectral_rolloff'
TON = 'tonnetz'
ZCR = 'zcr'
all_features = [CENS,CQT,STFT,MFCC,RMSE,BW,CENT,CONT,ROLLOFF,TON,ZCR]

class Ridge(object):
    def __init__(self,tracks,features):
        self.tracks = tracks
        self.features = features
        self.small = tracks['set', 'subset'] <= 'small'
        self.training = tracks['set', 'split'] == 'training'
        self.validation = tracks['set', 'split'] == 'validation'
        self.testing = tracks['set', 'split'] == 'test'

    def datasplit(self,feature_array):
        # takes an array of features [MFCC, CONT]
        X_train_temp = self.features.loc[self.small & (self.training | self.validation), feature_array]
        X_test_temp = self.features.loc[self.small & self.testing, feature_array]
        y_train_temp = self.tracks.loc[self.small & (self.training | self.validation), ('track', 'genre_top')]
        y_test_temp = self.tracks.loc[self.small & self.testing, ('track', 'genre_top')]
        y_train = y_train_temp.dropna()
        y_test = y_test_temp.dropna()
        X_train = X_train_temp.drop(y_train_temp.drop(y_train.index).index)
        X_test = X_test_temp.drop(y_test_temp.drop(y_test.index).index)
        EXPERIMENTAL = self.tracks['track', 'genre_top'] == "Experimental"
        X_train = X_train.drop(X_train.loc[EXPERIMENTAL].index)
        y_train = y_train.drop(y_train.loc[EXPERIMENTAL].index)
        X_test = X_test.drop(X_test.loc[EXPERIMENTAL].index)
        y_test = y_test.drop(y_test.loc[EXPERIMENTAL].index)
        return skl.utils.shuffle(X_train, y_train, random_state=42), X_test, y_test

    def train(self):
        (X_train, y_train), X_test, y_test = self.datasplit(all_features)
        self.ridge = skl.linear_model.RidgeClassifierCV().fit(X_train, y_train)
        print("Training Report")
        print(sklearn.metrics.classification_report(y_train, self.ridge.predict(X_train)))
        print()
        print("Test Report")
        print(sklearn.metrics.classification_report(y_test, self.ridge.predict(X_test)))
        
    def test(self, audio_feature):
        print(self.ridge.predict(audio_feature))
        
class SVM(object):
    def __init__(self,tracks,features):
        self.tracks = tracks
        self.features = features
        self.small = tracks['set', 'subset'] <= 'small'
        self.training = tracks['set', 'split'] == 'training'
        self.validation = tracks['set', 'split'] == 'validation'
        self.testing = tracks['set', 'split'] == 'test'
    
    
    def datasplit(self,feature_array):
        # takes an array of features [MFCC, CONT]
        X_train_temp = self.features.loc[self.small & (self.training | self.validation), feature_array]
        X_test_temp = self.features.loc[self.small & self.testing, feature_array]
        y_train_temp = self.tracks.loc[self.small & (self.training | self.validation), ('track', 'genre_top')]
        y_test_temp = self.tracks.loc[self.small & self.testing, ('track', 'genre_top')]
        y_train = y_train_temp.dropna()
        y_test = y_test_temp.dropna()
        X_train = X_train_temp.drop(y_train_temp.drop(y_train.index).index)
        X_test = X_test_temp.drop(y_test_temp.drop(y_test.index).index)
        EXPERIMENTAL = self.tracks['track', 'genre_top'] == "Experimental"
        X_train = X_train.drop(X_train.loc[EXPERIMENTAL].index)
        y_train = y_train.drop(y_train.loc[EXPERIMENTAL].index)
        X_test = X_test.drop(X_test.loc[EXPERIMENTAL].index)
        y_test = y_test.drop(y_test.loc[EXPERIMENTAL].index)
        return skl.utils.shuffle(X_train, y_train, random_state=42), X_test, y_test

    def train(self, feature_array, c, g):
        (X_train, y_train), X_test, y_test = self.datasplit(feature_array)
        scaler = skl.preprocessing.StandardScaler(copy=False)
        scaler.fit_transform(X_train)
        scaler.transform(X_test)
        self.ridge = skl.svm.SVC(kernel = "rbf", C = c, gamma = g).fit(X_train, y_train)
        #print("Training Report")
        print("Train Accuracy: ", self.ridge.score(X_train, y_train))
        print()
        #print(sklearn.metrics.classification_report(y_train, self.ridge.predict(X_train)))
        print()
        #print("Test Report")
        print("Test Accuracy: ", self.ridge.score(X_test, y_test))
        #print(sklearn.metrics.classification_report(y_test, self.ridge.predict(X_test)))
        
    def test(self, audio_feature):
        print(self.ridge.predict(audio_feature))
        
    def cv(self, feature_array):
        (X_train, y_train), X_test, y_test = self.datasplit(feature_array)
        scaler = skl.preprocessing.StandardScaler(copy=False)
        scaler.fit_transform(X_train)
        scaler.transform(X_test)
        tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-1, 1e0, 1e1],
                             'C': [1, 10]},
                            {'kernel': ['linear'], 'C': [1, 10]}]

        scores = ['precision', 'recall']

        for score in scores:
            print("# Tuning hyper-parameters for %s" % score)
            print()

            clf = skl.model_selection.GridSearchCV(skl.svm.SVC(), tuned_parameters, cv=3, scoring='%s_macro' % score)
            clf.fit(X_train, y_train)

            print("Best parameters set found on development set:")
            print()
            print(clf.best_params_)
            print()
            print("Grid scores on development set:")
            print()
            means = clf.cv_results_['mean_test_score']
            stds = clf.cv_results_['std_test_score']
            for mean, std, params in zip(means, stds, clf.cv_results_['params']):
                print("%0.3f (+/-%0.03f) for %r"
                      % (mean, std * 2, params))
            print()

            print("Detailed classification report:")
            print()
            print("The model is trained on the full development set.")
            print("The scores are computed on the full evaluation set.")
            print()
            y_true, y_pred = y_test, clf.predict(X_test)
            print(classification_report(y_true, y_pred))
            print()
            
class LR(object):
    def __init__(self,tracks,features):
        self.tracks = tracks
        self.features = features
        self.small = tracks['set', 'subset'] <= 'small'
        self.training = tracks['set', 'split'] == 'training'
        self.validation = tracks['set', 'split'] == 'validation'
        self.testing = tracks['set', 'split'] == 'test'
    
    
    def datasplit(self,feature_array):
        # takes an array of features [MFCC, CONT]
        X_train_temp = self.features.loc[self.small & (self.training | self.validation), feature_array]
        X_test_temp = self.features.loc[self.small & self.testing, feature_array]
        y_train_temp = self.tracks.loc[self.small & (self.training | self.validation), ('track', 'genre_top')]
        y_test_temp = self.tracks.loc[self.small & self.testing, ('track', 'genre_top')]
        y_train = y_train_temp.dropna()
        y_test = y_test_temp.dropna()
        X_train = X_train_temp.drop(y_train_temp.drop(y_train.index).index)
        X_test = X_test_temp.drop(y_test_temp.drop(y_test.index).index)
        EXPERIMENTAL = self.tracks['track', 'genre_top'] == "Experimental"
        X_train = X_train.drop(X_train.loc[EXPERIMENTAL].index)
        y_train = y_train.drop(y_train.loc[EXPERIMENTAL].index)
        X_test = X_test.drop(X_test.loc[EXPERIMENTAL].index)
        y_test = y_test.drop(y_test.loc[EXPERIMENTAL].index)
        return skl.utils.shuffle(X_train, y_train, random_state=42), X_test, y_test

    def train(self, feature_array):
        (X_train, y_train), X_test, y_test = self.datasplit(feature_array)
        self.lr = skl.linear_model.LogisticRegression().fit(X_train, y_train)
        print("Training Report")
        print("Train Accuracy: ", self.lr.score(X_train, y_train))
        print(sklearn.metrics.classification_report(y_train, self.lr.predict(X_train)))
        print()
        print("Test Report")
        print("Test Accuracy: ", self.ridge.score(X_test, y_test))
        print(sklearn.metrics.classification_report(y_test, self.lr.predict(X_test)))
        
    def test(self, audio_feature):
        print(self.ridge.predict(audio_feature))
        
    def cv(self, feature_array):
        (X_train, y_train), X_test, y_test = self.datasplit(feature_array)
        
        #[0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
        tuned_parameters = [{"C":np.logspace(-3,3,7), "penalty":["l1","l2"], "solver":['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}]

        scores = ['precision', 'recall']

        for score in scores:
            print("# Tuning hyper-parameters for %s" % score)
            print()

            clf = skl.model_selection.GridSearchCV(skl.linear_model.LogisticRegression(), tuned_parameters, cv=5, scoring='%s_macro' % score)
            clf.fit(X_train, y_train)

            print("Best parameters set found on development set:")
            print()
            print(clf.best_params_)
            print()
            print("Grid scores on development set:")
            print()
            means = clf.cv_results_['mean_test_score']
            stds = clf.cv_results_['std_test_score']
            for mean, std, params in zip(means, stds, clf.cv_results_['params']):
                print("%0.3f (+/-%0.03f) for %r"
                      % (mean, std * 2, params))
            print()

            print("Detailed classification report:")
            print()
            print("The model is trained on the full development set.")
            print("The scores are computed on the full evaluation set.")
            print()
            y_true, y_pred = y_test, clf.predict(X_test)
            print(classification_report(y_true, y_pred))
            print()

In [4]:
rgd = Ridge(tracks, features)
rgd.train()

Training Report


C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       0.00      0.00      0.00        97
          Classical       0.75      0.79      0.77      1143
            Country       0.00      0.00      0.00       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.64      0.82      0.72      8533
               Folk       0.61      0.56      0.58      2504
            Hip-Hop       0.69      0.57      0.63      3229
       Instrumental       0.65      0.17      0.27      1770
      International       0.85      0.17      0.28      1261
               Jazz       1.00      0.02      0.03       524
Old-Time / Historic       0.88      0.90      0.89       499
                Pop       0.57      0.03      0.05      2128
               Rock       0.69      0.91      0.78     12718
           Soul-RnB       0.00      0.00      0.00       132
             Spoken       0.56      0.14      0.23       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
audio_features = compute_features('data/080517.wav')
rgd.test(audio_features)

['Rock']


In [ ]:
svm = SVM(tracks, features)
svm.cv([MFCC, CENT, CONT])

# Tuning hyper-parameters for precision



In [5]:
svm = SVM(tracks, features)
svm.train([MFCC, CONT])

Training Report


C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       1.00      0.18      0.30        97
          Classical       0.93      0.89      0.91      1143
            Country       1.00      0.13      0.23       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.76      0.91      0.83      8533
               Folk       0.77      0.75      0.76      2504
            Hip-Hop       0.84      0.74      0.79      3229
       Instrumental       0.79      0.55      0.65      1770
      International       0.89      0.61      0.72      1261
               Jazz       0.90      0.45      0.60       524
Old-Time / Historic       1.00      0.99      0.99       499
                Pop       0.87      0.33      0.48      2128
               Rock       0.82      0.93      0.88     12718
           Soul-RnB       1.00      0.02      0.03       132
             Spoken       0.79      0.77      0.78       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# FEATURE SELECTION

CENS = 'chroma_cens'
CQT = 'chroma_cqt'
STFT = 'chroma_stft'
MFCC = 'mfcc'
RMSE = 'rmse'
BW = 'spectral_bandwidth'
CENT = 'spectral_centroid'
CONT = 'spectral_contrast'
ROLLOFF = 'spectral_rolloff'
TON = 'tonnetz'
ZCR = 'zcr'

In [11]:
#rbf c = 1.5
svm = SVM(tracks, features)
svm.train([MFCC, CONT])

Training Report
0.846543673841



C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       1.00      0.33      0.50        97
          Classical       0.95      0.92      0.94      1143
            Country       1.00      0.31      0.48       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.80      0.93      0.86      8533
               Folk       0.81      0.79      0.80      2504
            Hip-Hop       0.87      0.79      0.83      3229
       Instrumental       0.85      0.64      0.73      1770
      International       0.92      0.67      0.78      1261
               Jazz       0.94      0.60      0.73       524
Old-Time / Historic       1.00      0.99      1.00       499
                Pop       0.92      0.45      0.60      2128
               Rock       0.85      0.95      0.90     12718
           Soul-RnB       1.00      0.13      0.23       132
             Spoken       0.84      0.85      0.85       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [12]:
#rbf c = 1.5
print("MFCC ONLY")
svm = SVM(tracks, features)
svm.train([MFCC])

MFCC ONLY
Training Report
0.813944880993



C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       1.00      0.28      0.44        97
          Classical       0.93      0.90      0.92      1143
            Country       1.00      0.23      0.38       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.75      0.91      0.82      8533
               Folk       0.78      0.75      0.76      2504
            Hip-Hop       0.84      0.74      0.79      3229
       Instrumental       0.79      0.57      0.66      1770
      International       0.90      0.61      0.73      1261
               Jazz       0.92      0.50      0.65       524
Old-Time / Historic       0.99      0.99      0.99       499
                Pop       0.91      0.36      0.51      2128
               Rock       0.83      0.93      0.88     12718
           Soul-RnB       1.00      0.07      0.13       132
             Spoken       0.84      0.78      0.81       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
#rbf c = 1.5
print("MFCC CENT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT])

MFCC CENT
Training Report
0.816962760506



C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       1.00      0.29      0.45        97
          Classical       0.93      0.91      0.92      1143
            Country       1.00      0.26      0.41       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.76      0.91      0.83      8533
               Folk       0.78      0.75      0.77      2504
            Hip-Hop       0.83      0.75      0.79      3229
       Instrumental       0.81      0.57      0.67      1770
      International       0.91      0.62      0.74      1261
               Jazz       0.91      0.51      0.65       524
Old-Time / Historic       0.99      0.99      0.99       499
                Pop       0.89      0.37      0.52      2128
               Rock       0.83      0.93      0.88     12718
           Soul-RnB       1.00      0.08      0.14       132
             Spoken       0.84      0.78      0.81       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
#rbf c = 1.5
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT])

MFCC CENT CONT
Training Report
0.849106024371



C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       1.00      0.34      0.51        97
          Classical       0.96      0.92      0.94      1143
            Country       1.00      0.34      0.50       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.80      0.93      0.86      8533
               Folk       0.81      0.80      0.80      2504
            Hip-Hop       0.87      0.79      0.83      3229
       Instrumental       0.86      0.65      0.74      1770
      International       0.93      0.67      0.78      1261
               Jazz       0.94      0.61      0.74       524
Old-Time / Historic       1.00      0.99      1.00       499
                Pop       0.91      0.45      0.61      2128
               Rock       0.86      0.95      0.90     12718
           Soul-RnB       1.00      0.16      0.27       132
             Spoken       0.84      0.85      0.85       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
#rbf c = 1.5
print("MFCC CENT CENS CQT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CENS, CQT])

MFCC CENT CENS CQT
Training Report
0.833447215579



C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       1.00      0.20      0.33        97
          Classical       0.93      0.92      0.93      1143
            Country       1.00      0.19      0.32       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.79      0.92      0.85      8533
               Folk       0.81      0.78      0.80      2504
            Hip-Hop       0.86      0.78      0.82      3229
       Instrumental       0.88      0.65      0.75      1770
      International       0.91      0.62      0.74      1261
               Jazz       0.91      0.55      0.68       524
Old-Time / Historic       1.00      0.98      0.99       499
                Pop       0.92      0.38      0.54      2128
               Rock       0.84      0.94      0.89     12718
           Soul-RnB       1.00      0.05      0.10       132
             Spoken       0.77      0.74      0.76       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [16]:
#rbf c = 1.5
print("MFCC CONT CENS CQT")
svm = SVM(tracks, features)
svm.train([MFCC, CONT, CENS, CQT])

MFCC CONT CENS CQT
Training Report
0.854942489466



C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       1.00      0.29      0.45        97
          Classical       0.95      0.94      0.94      1143
            Country       1.00      0.25      0.40       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.82      0.94      0.87      8533
               Folk       0.83      0.81      0.82      2504
            Hip-Hop       0.87      0.82      0.84      3229
       Instrumental       0.90      0.67      0.77      1770
      International       0.92      0.68      0.78      1261
               Jazz       0.92      0.60      0.73       524
Old-Time / Historic       1.00      0.99      0.99       499
                Pop       0.92      0.44      0.60      2128
               Rock       0.85      0.95      0.90     12718
           Soul-RnB       1.00      0.09      0.17       132
             Spoken       0.80      0.82      0.81       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [17]:
#rbf c = 1.5
print("MFCC CONT CENT CENS CQT")
svm = SVM(tracks, features)
svm.train([MFCC, CONT, CENT, CENS, CQT])

MFCC CONT CENT CENS CQT
Training Report
0.857220134381



C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       1.00      0.29      0.45        97
          Classical       0.95      0.94      0.94      1143
            Country       1.00      0.26      0.41       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.82      0.94      0.88      8533
               Folk       0.83      0.81      0.82      2504
            Hip-Hop       0.88      0.82      0.84      3229
       Instrumental       0.91      0.68      0.77      1770
      International       0.92      0.69      0.78      1261
               Jazz       0.93      0.61      0.74       524
Old-Time / Historic       1.00      0.99      0.99       499
                Pop       0.92      0.45      0.60      2128
               Rock       0.86      0.95      0.90     12718
           Soul-RnB       1.00      0.11      0.19       132
             Spoken       0.81      0.83      0.82       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [18]:
#rbf c = 1.5
print("MFCC TON")
svm = SVM(tracks, features)
svm.train([MFCC, TON])

MFCC TON
Training Report
0.843184147591



C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       1.00      0.31      0.47        97
          Classical       0.95      0.92      0.93      1143
            Country       1.00      0.27      0.43       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.79      0.92      0.85      8533
               Folk       0.82      0.79      0.81      2504
            Hip-Hop       0.86      0.78      0.82      3229
       Instrumental       0.85      0.66      0.74      1770
      International       0.92      0.65      0.76      1261
               Jazz       0.92      0.60      0.72       524
Old-Time / Historic       1.00      0.99      0.99       499
                Pop       0.92      0.43      0.59      2128
               Rock       0.85      0.94      0.90     12718
           Soul-RnB       1.00      0.13      0.23       132
             Spoken       0.86      0.84      0.85       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [19]:
#rbf c = 1.5
print("MFCC ZCR")
svm = SVM(tracks, features)
svm.train([MFCC, ZCR])

MFCC ZCR
Training Report
0.817873818472



C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       1.00      0.27      0.42        97
          Classical       0.94      0.91      0.92      1143
            Country       1.00      0.25      0.40       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.76      0.91      0.83      8533
               Folk       0.79      0.75      0.77      2504
            Hip-Hop       0.84      0.74      0.79      3229
       Instrumental       0.80      0.58      0.67      1770
      International       0.91      0.61      0.73      1261
               Jazz       0.91      0.51      0.65       524
Old-Time / Historic       0.99      0.99      0.99       499
                Pop       0.90      0.37      0.53      2128
               Rock       0.83      0.94      0.88     12718
           Soul-RnB       1.00      0.11      0.19       132
             Spoken       0.83      0.77      0.80       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [20]:
#rbf c = 1.5
print("MFCC CONT CENT TON")
svm = SVM(tracks, features)
svm.train([MFCC, CONT, CENT, TON])

MFCC CONT CENT TON
Training Report
0.866188361234

                     precision    recall  f1-score   support

              Blues       1.00      0.38      0.55        97
          Classical       0.97      0.94      0.95      1143
            Country       1.00      0.38      0.55       176
     Easy Listening       1.00      0.06      0.11        18
         Electronic       0.83      0.94      0.88      8533
               Folk       0.84      0.82      0.83      2504
            Hip-Hop       0.89      0.82      0.86      3229
       Instrumental       0.88      0.69      0.77      1770
      International       0.93      0.72      0.81      1261
               Jazz       0.96      0.67      0.79       524
Old-Time / Historic       1.00      0.99      1.00       499
                Pop       0.93      0.50      0.65      2128
               Rock       0.87      0.95      0.91     12718
           Soul-RnB       1.00      0.20      0.33       132
             Spoken       0.85   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [21]:
#rbf c = 1.5
print("MFCC CONT CENT ZCR")
svm = SVM(tracks, features)
svm.train([MFCC, CONT, CENT, ZCR])

MFCC CONT CENT ZCR
Training Report
0.850216376267



C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       1.00      0.34      0.51        97
          Classical       0.96      0.92      0.94      1143
            Country       1.00      0.35      0.52       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.81      0.93      0.86      8533
               Folk       0.82      0.79      0.80      2504
            Hip-Hop       0.87      0.80      0.83      3229
       Instrumental       0.86      0.65      0.74      1770
      International       0.93      0.67      0.78      1261
               Jazz       0.94      0.61      0.74       524
Old-Time / Historic       1.00      0.99      1.00       499
                Pop       0.91      0.45      0.61      2128
               Rock       0.86      0.95      0.90     12718
           Soul-RnB       1.00      0.19      0.32       132
             Spoken       0.84      0.85      0.85       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [22]:
#rbf c = 1.5
print("MFCC CONT CENT TON ZCR")
svm = SVM(tracks, features)
svm.train([MFCC, CONT, CENT, TON, ZCR])

MFCC CONT CENT TON ZCR
Training Report
0.868295182781

                     precision    recall  f1-score   support

              Blues       1.00      0.38      0.55        97
          Classical       0.97      0.94      0.95      1143
            Country       1.00      0.39      0.56       176
     Easy Listening       1.00      0.06      0.11        18
         Electronic       0.83      0.94      0.88      8533
               Folk       0.84      0.82      0.83      2504
            Hip-Hop       0.89      0.83      0.86      3229
       Instrumental       0.89      0.69      0.78      1770
      International       0.93      0.72      0.81      1261
               Jazz       0.96      0.67      0.79       524
Old-Time / Historic       1.00      0.99      1.00       499
                Pop       0.92      0.50      0.65      2128
               Rock       0.87      0.96      0.91     12718
           Soul-RnB       0.96      0.19      0.32       132
             Spoken       0.8

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [23]:
#rbf c = 1.5
print("MFCC CONT CENT CENS CQT TON")
svm = SVM(tracks, features)
svm.train([MFCC, CONT, CENT, CENS, CQT, TON])

MFCC CONT CENT CENS CQT TON
Training Report
0.863626010705



C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       1.00      0.31      0.47        97
          Classical       0.96      0.94      0.95      1143
            Country       1.00      0.27      0.43       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.83      0.94      0.88      8533
               Folk       0.84      0.82      0.83      2504
            Hip-Hop       0.88      0.82      0.85      3229
       Instrumental       0.91      0.69      0.79      1770
      International       0.91      0.70      0.79      1261
               Jazz       0.95      0.63      0.75       524
Old-Time / Historic       1.00      1.00      1.00       499
                Pop       0.92      0.47      0.63      2128
               Rock       0.86      0.96      0.91     12718
           Soul-RnB       1.00      0.14      0.24       132
             Spoken       0.82      0.84      0.83       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [24]:
#rbf c = 1.5
print("MFCC CONT CENT CENS CQT ZCR")
svm = SVM(tracks, features)
svm.train([MFCC, CONT, CENT, CENS, CQT, ZCR])

MFCC CONT CENT CENS CQT ZCR
Training Report
0.858842956383



C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       1.00      0.29      0.45        97
          Classical       0.95      0.94      0.94      1143
            Country       1.00      0.30      0.46       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.82      0.94      0.88      8533
               Folk       0.83      0.81      0.82      2504
            Hip-Hop       0.88      0.82      0.84      3229
       Instrumental       0.91      0.68      0.78      1770
      International       0.92      0.69      0.79      1261
               Jazz       0.93      0.62      0.74       524
Old-Time / Historic       1.00      0.99      1.00       499
                Pop       0.92      0.46      0.61      2128
               Rock       0.86      0.95      0.90     12718
           Soul-RnB       1.00      0.11      0.19       132
             Spoken       0.81      0.83      0.82       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [5]:
#rbf c = 1.5
print("MFCC CONT CENT CENS CQT TON ZCR")
svm = SVM(tracks, features)
svm.train([MFCC, CONT, CENT, CENS, CQT, TON, ZCR])

MFCC CONT CENT CENS CQT TON ZCR
Training Report
0.865476597198



C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       1.00      0.31      0.47        97
          Classical       0.95      0.94      0.95      1143
            Country       1.00      0.28      0.44       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.83      0.94      0.88      8533
               Folk       0.84      0.82      0.83      2504
            Hip-Hop       0.88      0.83      0.85      3229
       Instrumental       0.91      0.69      0.79      1770
      International       0.91      0.70      0.79      1261
               Jazz       0.94      0.63      0.76       524
Old-Time / Historic       1.00      0.99      0.99       499
                Pop       0.92      0.48      0.63      2128
               Rock       0.87      0.96      0.91     12718
           Soul-RnB       1.00      0.15      0.26       132
             Spoken       0.81      0.85      0.83       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [25]:
#rbf c = 1.5
print("CENS,CQT,STFT,MFCC,RMSE,BW,CENT,CONT,ROLLOFF,TON,ZCR")
svm = SVM(tracks, features)
svm.train([CENS,CQT,STFT,MFCC,RMSE,BW,CENT,CONT,ROLLOFF,TON,ZCR])

CENS,CQT,STFT,MFCC,RMSE,BW,CENT,CONT,ROLLOFF,TON,ZCR
Training Report
0.863882245758



C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       1.00      0.30      0.46        97
          Classical       0.95      0.94      0.94      1143
            Country       1.00      0.22      0.36       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.83      0.94      0.88      8533
               Folk       0.84      0.83      0.83      2504
            Hip-Hop       0.89      0.82      0.86      3229
       Instrumental       0.91      0.69      0.78      1770
      International       0.91      0.70      0.79      1261
               Jazz       0.94      0.62      0.75       524
Old-Time / Historic       1.00      0.99      0.99       499
                Pop       0.92      0.48      0.63      2128
               Rock       0.86      0.96      0.91     12718
           Soul-RnB       1.00      0.12      0.22       132
             Spoken       0.80      0.85      0.82       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [6]:
#rbf c = 1.5
print("MFCC CONT CENT CENS CQT STFT ROLLOFF")
svm = SVM(tracks, features)
svm.train([MFCC, CONT, CENT, CENS, CQT, TON, STFT, ROLLOFF])

MFCC CONT CENT CENS CQT STFT ROLLOFF
Training Report
0.859839426033



C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       1.00      0.29      0.45        97
          Classical       0.95      0.94      0.94      1143
            Country       1.00      0.22      0.36       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.82      0.94      0.88      8533
               Folk       0.83      0.82      0.83      2504
            Hip-Hop       0.88      0.82      0.85      3229
       Instrumental       0.91      0.68      0.78      1770
      International       0.91      0.69      0.78      1261
               Jazz       0.93      0.61      0.74       524
Old-Time / Historic       1.00      0.99      0.99       499
                Pop       0.93      0.46      0.62      2128
               Rock       0.86      0.95      0.90     12718
           Soul-RnB       1.00      0.12      0.22       132
             Spoken       0.79      0.83      0.81       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [7]:
#rbf c = 1.5
print("MFCC CONT CENT STFT ROLLOFF")
svm = SVM(tracks, features)
svm.train([MFCC, CONT, CENT, STFT, ROLLOFF])

MFCC CONT CENT STFT ROLLOFF
Training Report
0.843611206013



C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       1.00      0.32      0.48        97
          Classical       0.96      0.92      0.94      1143
            Country       1.00      0.28      0.44       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.80      0.93      0.86      8533
               Folk       0.81      0.79      0.80      2504
            Hip-Hop       0.86      0.79      0.82      3229
       Instrumental       0.87      0.64      0.73      1770
      International       0.92      0.65      0.76      1261
               Jazz       0.92      0.59      0.72       524
Old-Time / Historic       1.00      0.99      1.00       499
                Pop       0.91      0.43      0.58      2128
               Rock       0.85      0.95      0.89     12718
           Soul-RnB       1.00      0.07      0.13       132
             Spoken       0.81      0.81      0.81       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
#rbf c = 1.5
print("MFCC CONT CENT STFT")
svm = SVM(tracks, features)
svm.train([MFCC, CONT, CENT, STFT])

MFCC CONT CENT STFT
Training Report
0.843810499943



C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       1.00      0.30      0.46        97
          Classical       0.96      0.92      0.94      1143
            Country       1.00      0.28      0.44       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.80      0.93      0.86      8533
               Folk       0.81      0.80      0.80      2504
            Hip-Hop       0.86      0.79      0.82      3229
       Instrumental       0.87      0.64      0.74      1770
      International       0.92      0.65      0.76      1261
               Jazz       0.92      0.58      0.71       524
Old-Time / Historic       1.00      0.99      0.99       499
                Pop       0.90      0.43      0.58      2128
               Rock       0.85      0.95      0.89     12718
           Soul-RnB       1.00      0.08      0.14       132
             Spoken       0.82      0.81      0.82       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
#rbf c = 1.5
print("MFCC CONT CENT ROLLOFF")
svm = SVM(tracks, features)
svm.train([MFCC, CONT, CENT, ROLLOFF])

MFCC CONT CENT ROLLOFF
Training Report
0.848365789773



C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       1.00      0.38      0.55        97
          Classical       0.96      0.92      0.94      1143
            Country       1.00      0.33      0.50       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.80      0.93      0.86      8533
               Folk       0.81      0.79      0.80      2504
            Hip-Hop       0.87      0.79      0.83      3229
       Instrumental       0.85      0.65      0.74      1770
      International       0.93      0.67      0.78      1261
               Jazz       0.94      0.60      0.73       524
Old-Time / Historic       1.00      0.99      1.00       499
                Pop       0.91      0.45      0.60      2128
               Rock       0.85      0.95      0.90     12718
           Soul-RnB       1.00      0.15      0.26       132
             Spoken       0.84      0.85      0.84       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
#rbf c = 1.5
print("MFCC CONT CENT CENS CQT TON CQT STFT ROLLOFF")
svm = SVM(tracks, features)
svm.train([MFCC, CONT, CENT, CENS, CQT, TON, CQT,  STFT, ROLLOFF])

MFCC CONT CENT CENS CQT TON CQT STFT ROLLOFF
Training Report
0.859839426033



C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                     precision    recall  f1-score   support

              Blues       1.00      0.29      0.45        97
          Classical       0.95      0.94      0.94      1143
            Country       1.00      0.22      0.36       176
     Easy Listening       0.00      0.00      0.00        18
         Electronic       0.82      0.94      0.88      8533
               Folk       0.83      0.82      0.83      2504
            Hip-Hop       0.88      0.82      0.85      3229
       Instrumental       0.91      0.68      0.78      1770
      International       0.91      0.69      0.78      1261
               Jazz       0.93      0.61      0.74       524
Old-Time / Historic       1.00      0.99      0.99       499
                Pop       0.93      0.46      0.62      2128
               Rock       0.86      0.95      0.90     12718
           Soul-RnB       1.00      0.12      0.22       132
             Spoken       0.79      0.83      0.81       392

          micro avg   

C:\Users\Msi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 1, 1e-2)

In [13]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 10, 1e-2)

MFCC CENT CONT
Train Accuracy:  0.998661883612


Test Accuracy:  0.653388515261


In [14]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 100, 1e-2)

MFCC CENT CONT
Train Accuracy:  0.99880423642


Test Accuracy:  0.653905845835


In [ ]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 1000, 1e-2)

MFCC CENT CONT


In [9]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 1, 1e-3)

MFCC CENT CONT
Train Accuracy:  0.705329689101


Test Accuracy:  0.660889808588


In [10]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 10, 1e-3)

MFCC CENT CONT
Train Accuracy:  0.792193372053


Test Accuracy:  0.670460424211


In [11]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 100, 1e-3)

MFCC CENT CONT
Train Accuracy:  0.935770413393


Test Accuracy:  0.6526125194


In [12]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 1000, 1e-3)

MFCC CENT CONT
Train Accuracy:  0.998377177998


Test Accuracy:  0.627521986549


In [5]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 1, 1e-4)

MFCC CENT CONT
Train Accuracy:  0.647505978818


Test Accuracy:  0.626228660114


In [6]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 10, 1e-4)

MFCC CENT CONT
Train Accuracy:  0.690040997608


Test Accuracy:  0.653647180548


In [7]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 100, 1e-4)

MFCC CENT CONT
Train Accuracy:  0.725856963899


Test Accuracy:  0.66244180031


In [8]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 1000, 1e-4)

MFCC CENT CONT
Train Accuracy:  0.800279011502


Test Accuracy:  0.66244180031


In [8]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 8, 1e-3)

MFCC CENT CONT
Train Accuracy:  0.780577382986


Test Accuracy:  0.668908432488


In [9]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 15, 1e-3)

MFCC CENT CONT
Train Accuracy:  0.814571233345


Test Accuracy:  0.669943093637


In [10]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 20, 1e-3)

MFCC CENT CONT
Train Accuracy:  0.832023687507


Test Accuracy:  0.668132436627


In [12]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 25, 1e-3)

MFCC CENT CONT
Train Accuracy:  0.846458262157


Test Accuracy:  0.668391101914


In [13]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 40, 1e-3)

MFCC CENT CONT
Train Accuracy:  0.877348821319


Test Accuracy:  0.666580444904


In [14]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 60, 1e-4)

MFCC CENT CONT
Train Accuracy:  0.716233914133


Test Accuracy:  0.659079151578


In [15]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 1, 1e-5)

MFCC CENT CONT
Train Accuracy:  0.541196902403


Test Accuracy:  0.538023797206


In [16]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 5, 1e-5)

MFCC CENT CONT
Train Accuracy:  0.626039175493


Test Accuracy:  0.606570098293


In [17]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 10, 1e-5)

MFCC CENT CONT
Train Accuracy:  0.644658922674


Test Accuracy:  0.62390067253


In [18]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 50, 1e-5)

MFCC CENT CONT
Train Accuracy:  0.675264776221


Test Accuracy:  0.648215209519


In [19]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 1, 1e-1)

MFCC CENT CONT
Train Accuracy:  0.998604942489


Test Accuracy:  0.379203310916


In [20]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 1.2, 1e-1)

MFCC CENT CONT
Train Accuracy:  0.998690354174


Test Accuracy:  0.379203310916


In [21]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 1.5, 1e-1)

MFCC CENT CONT
Train Accuracy:  0.998747295297


Test Accuracy:  0.379203310916


In [22]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 0.8, 1e-1)

MFCC CENT CONT
Train Accuracy:  0.709344038264


Test Accuracy:  0.379203310916


In [23]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 5, 1e-1)

MFCC CENT CONT
Train Accuracy:  0.99880423642


Test Accuracy:  0.379203310916


In [24]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 0.8, "auto_deprecated")

MFCC CENT CONT
Train Accuracy:  0.791567019702


Test Accuracy:  0.673823072944


In [25]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 0.9, "auto_deprecated")

MFCC CENT CONT
Train Accuracy:  0.801987245188


Test Accuracy:  0.673564407656


In [26]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 1.1, "auto_deprecated")

MFCC CENT CONT
Train Accuracy:  0.821147933037


Test Accuracy:  0.676409725815


In [27]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 1.2, "auto_deprecated")

MFCC CENT CONT
Train Accuracy:  0.828180161713


Test Accuracy:  0.677961717538


In [28]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 1.3, "auto_deprecated")

MFCC CENT CONT
Train Accuracy:  0.835724860494


Test Accuracy:  0.67770305225


In [29]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 1.4, "auto_deprecated")

MFCC CENT CONT
Train Accuracy:  0.842842500854


Test Accuracy:  0.677444386963


In [30]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 1.45, "auto_deprecated")

MFCC CENT CONT
Train Accuracy:  0.845746498121


Test Accuracy:  0.676927056389


In [31]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 1.55, "auto_deprecated")

MFCC CENT CONT
Train Accuracy:  0.852351668375


Test Accuracy:  0.676151060528


# THE HOPEFUL ONES

In [5]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 1.22, "auto_deprecated")

MFCC CENT CONT
Train Accuracy:  0.830059218768


Test Accuracy:  0.67770305225


In [ ]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 1.18, "auto_deprecated")

In [5]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 1.19, "auto_deprecated")

MFCC CENT CONT
Train Accuracy:  0.827639221045


Test Accuracy:  0.677444386963


In [6]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 1.21, "auto_deprecated")

MFCC CENT CONT
Train Accuracy:  0.829233572486


Test Accuracy:  0.67770305225


In [7]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 12, 1e-3)

MFCC CENT CONT
Train Accuracy:  0.802072656873


Test Accuracy:  0.672012415934


In [8]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 11, 1e-3)

MFCC CENT CONT
Train Accuracy:  0.796834073568


Test Accuracy:  0.67149508536


In [9]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 11.5, 1e-3)

MFCC CENT CONT
Train Accuracy:  0.799738070835


Test Accuracy:  0.672788411795


In [10]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 150, 1e-5)

MFCC CENT CONT
Train Accuracy:  0.689642409748


Test Accuracy:  0.651319192964


In [11]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 300, 1e-5)

MFCC CENT CONT
Train Accuracy:  0.696959344038


Test Accuracy:  0.656492498707


In [ ]:
print("MFCC CENT CONT")
svm = SVM(tracks, features)
svm.train([MFCC, CENT, CONT], 82000, 1e-5)

MFCC CENT CONT
